In [1]:
import pandas as pd
import os
import requests
import json
from bs4 import BeautifulSoup as b

import ast

from sqlalchemy import create_engine




In [2]:
file='traffic-collision-data-from-2010-to-present.csv'

In [3]:
df= pd.read_csv(file, low_memory=False, parse_dates=['Date Reported'])

In [4]:
df.columns

Index(['DR Number', 'Date Reported', 'Date Occurred', 'Time Occurred',
       'Area ID', 'Area Name', 'Reporting District', 'Crime Code',
       'Crime Code Description', 'MO Codes', 'Victim Age', 'Victim Sex',
       'Victim Descent', 'Premise Code', 'Premise Description', 'Address',
       'Cross Street', 'Location', 'Zip Codes', 'Census Tracts',
       'Precinct Boundaries', 'LA Specific Plans', 'Council Districts',
       'Neighborhood Councils (Certified)'],
      dtype='object')

In [5]:
len(df)


477377

In [6]:
#df.head(n)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 477377 entries, 0 to 477376
Data columns (total 24 columns):
DR Number                            477377 non-null int64
Date Reported                        477377 non-null datetime64[ns]
Date Occurred                        477377 non-null object
Time Occurred                        477377 non-null int64
Area ID                              477377 non-null int64
Area Name                            477377 non-null object
Reporting District                   477377 non-null int64
Crime Code                           477377 non-null int64
Crime Code Description               477377 non-null object
MO Codes                             392557 non-null object
Victim Age                           399952 non-null float64
Victim Sex                           470422 non-null object
Victim Descent                       469708 non-null object
Premise Code                         477352 non-null float64
Premise Description                  477352 

In [28]:
df.iloc[0]['Location']


'{\'latitude\': \'34.2279\', \'human_address\': \'{"address": "", "city": "", "state": "", "zip": ""}\', \'longitude\': \'-118.4516\'}'

In [30]:
#The fix for all the unusual quotes
# imported ast above
ast.literal_eval(df.iloc[0]['Location'])

{'human_address': '{"address": "", "city": "", "state": "", "zip": ""}',
 'latitude': '34.2279',
 'longitude': '-118.4516'}

In [ ]:
#Above looks like a normal dictionary now

In [8]:
df['date_stamp'] =  pd.to_datetime(df['Date Occurred'], format='%Y-%m-%dT%H:%M:%S')
df.shape

(477377, 25)

In [9]:
df['year'] = pd.DatetimeIndex(df['date_stamp']).year
df['month'] = pd.DatetimeIndex(df['date_stamp']).month
df['day'] = pd.DatetimeIndex(df['date_stamp']).day
df['date'] = pd.DatetimeIndex(df['date_stamp']).date
df['weekday']= pd.DatetimeIndex(df['date_stamp']).dayofweek
df.shape

(477377, 30)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 477377 entries, 0 to 477376
Data columns (total 30 columns):
DR Number                            477377 non-null int64
Date Reported                        477377 non-null datetime64[ns]
Date Occurred                        477377 non-null object
Time Occurred                        477377 non-null int64
Area ID                              477377 non-null int64
Area Name                            477377 non-null object
Reporting District                   477377 non-null int64
Crime Code                           477377 non-null int64
Crime Code Description               477377 non-null object
MO Codes                             392557 non-null object
Victim Age                           399952 non-null float64
Victim Sex                           470422 non-null object
Victim Descent                       469708 non-null object
Premise Code                         477352 non-null float64
Premise Description                  477352 

In [11]:
df2=df[['Area ID', 'Area Name', 'Date Reported', 'Time Occurred',\
        'Victim Age', 'Victim Sex', 'Zip Codes',\
       'Location', 'Premise Description', 'date_stamp', 'year', 'month', 'day', 'date', 'weekday']]

In [12]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 477377 entries, 0 to 477376
Data columns (total 15 columns):
Area ID                477377 non-null int64
Area Name              477377 non-null object
Date Reported          477377 non-null datetime64[ns]
Time Occurred          477377 non-null int64
Victim Age             399952 non-null float64
Victim Sex             470422 non-null object
Zip Codes              476984 non-null float64
Location               477377 non-null object
Premise Description    477352 non-null object
date_stamp             477377 non-null datetime64[ns]
year                   477377 non-null int64
month                  477377 non-null int64
day                    477377 non-null int64
date                   477377 non-null object
weekday                477377 non-null int64
dtypes: datetime64[ns](2), float64(2), int64(6), object(5)
memory usage: 54.6+ MB


In [13]:
#df2['Premise Description'].unique()
#df2['Premise Description'].value_counts()

In [14]:
#df2['Location'][0]

In [15]:
df3 = df2.copy()

In [16]:
#removes strange quotes
df2['Location'] = [ast.literal_eval(d) for d in df2.Location]

/Users/williamkapsalis/anaconda/envs/PythonData/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [17]:
df3['longitude'] = [d['longitude'] for d in df2.Location]

In [18]:
df3['latitude'] = [d['latitude'] for d in df2.Location]

In [19]:
df3.head(n=2)

,Area ID,Area Name,Date Reported,Time Occurred,Victim Age,Victim Sex,Zip Codes,Location,Premise Description,date_stamp,year,month,day,date,weekday,longitude,latitude
0,19,Mission,2019-06-22,1150,32.0,M,19730.0,"{'latitude': '34.2279', 'human_address': '{""ad...",STREET,2019-06-22,2019,6,22,2019-06-22,5,-118.4516,34.2279
1,3,Southwest,2019-06-22,1600,27.0,M,22724.0,"{'latitude': '34.0255', 'human_address': '{""ad...",STREET,2019-06-22,2019,6,22,2019-06-22,5,-118.2937,34.0255


In [20]:
df3.rename(columns={'Area ID':'ID', 'Area Name': 'Areaname' ,'Date Reported': 'DateReported' ,'date' : 'Date' ,'Time Occurred': 'Time' ,'Victim Age':'Age' ,'Victim Sex':'Sex' ,'Zip Codes':'ZipCode','Premise Description':'PremiseDescription','year':'Year' ,'day':'Day','longitude':'Longitude' ,'latitude':'Latitude'},inplace= True)
                    
                    
                    

In [21]:
df3.head(n=2)

,ID,Areaname,DateReported,Time,Age,Sex,ZipCode,Location,PremiseDescription,date_stamp,Year,month,Day,Date,weekday,Longitude,Latitude
0,19,Mission,2019-06-22,1150,32.0,M,19730.0,"{'latitude': '34.2279', 'human_address': '{""ad...",STREET,2019-06-22,2019,6,22,2019-06-22,5,-118.4516,34.2279
1,3,Southwest,2019-06-22,1600,27.0,M,22724.0,"{'latitude': '34.0255', 'human_address': '{""ad...",STREET,2019-06-22,2019,6,22,2019-06-22,5,-118.2937,34.0255


In [22]:
df3.shape

(477377, 17)

In [23]:
clean_df3= df3.dropna(how="any")
clean_df3.count()


ID                    398928
Areaname              398928
DateReported          398928
Time                  398928
Age                   398928
Sex                   398928
ZipCode               398928
Location              398928
PremiseDescription    398928
date_stamp            398928
Year                  398928
month                 398928
Day                   398928
Date                  398928
weekday               398928
Longitude             398928
Latitude              398928
dtype: int64

In [24]:

df3["ID"].unique()


array([19,  3, 10, 21, 15, 17,  9, 16, 20,  1, 11, 14,  5,  7,  2, 13,  8,
       12,  4, 18,  6])

In [25]:
df3['Areaname'].value_counts()

77th Street    30036
Southwest      28037
Wilshire       27446
West LA        26010
N Hollywood    25469
Olympic        25423
Pacific        25174
Van Nuys       24169
Newton         23591
Devonshire     23562
Hollywood      23357
West Valley    23222
Northeast      21926
Mission        20901
Topanga        20802
Central        19328
Southeast      19214
Rampart        17954
Harbor         17811
Hollenbeck     17379
Foothill       16566
Name: Areaname, dtype: int64

In [26]:
df3['PremiseDescription'].value_counts()

STREET                                                       455351
PARKING LOT                                                   15446
SIDEWALK                                                       2382
ALLEY                                                           826
DRIVEWAY                                                        806
FREEWAY                                                         444
SINGLE FAMILY DWELLING                                          316
GAS STATION                                                     302
TRANSPORTATION FACILITY (AIRPORT)                               154
GARAGE/CARPORT                                                  135
OTHER PREMISE                                                   129
OTHER/OUTSIDE                                                   101
VACANT LOT                                                       93
PARK/PLAYGROUND                                                  90
MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)    